In [1]:
import numpy as np

In [17]:
class FeatureComputation:
    def __init__(self, X, X_base, N_win, N_step, alpha, beta, N_d_point, N_d_win):
        """
        Initialize the feature computation process with necessary parameters.
        
        Parameters:
            X (ndarray): Input spatio-temporal data, shape (L, T)
            X_base (float): Background noise data for normalization.
            N_win (int): Window size for time dimension.
            N_step (int): Step size for sliding window.
            alpha (int): Step for energy computation.
            beta (float): Threshold for peak count.
            N_d_point (int): Spatial window size.
            N_d_win (int): Temporal window size.
        """
        self.X = X
        self.X_base = X_base
        self.N_win = N_win
        self.N_step = N_step
        self.alpha = alpha
        self.beta = beta
        self.N_d_point = N_d_point
        self.N_d_win = N_d_win
        self.L, self.T = X.shape  # L: spatial points, T: temporal points
        self.num_windows = (self.T - self.N_win) // self.N_step + 1  # Total number of sliding windows
    
    def count_peaks(self, data_segment, threshold=0):
        """
        Count the number of peaks in a data segment above a threshold.
        A peak is defined where the signal increases and then decreases.
        """
        peaks = 0
        for i in range(1, len(data_segment) - 1):
            if data_segment[i] > threshold and data_segment[i] > data_segment[i - 1] and data_segment[i] > data_segment[i + 1]:
                peaks += 1
        return peaks

    def compute(self):
        """
        Compute F_peak and F_energy matrices from input data and then aggregate M_peak and M_energy.
        
        Returns:
            M_peak (ndarray): Aggregated matrix of Peak Features.
            M_energy (ndarray): Aggregated matrix of Energy Features.
        """
        # Step 1: Attenuation compensation and standardization
        X = self.X / self.X_base  # Normalize input data
        
        # Initialize matrices
        F_peak = np.zeros((self.L, self.num_windows))
        F_energy = np.zeros((self.L, self.num_windows))

        # Step 2–10: Sliding window computations
        for i in range(self.L):  # Iterate over spatial points
            for j in range(self.num_windows):  # Sliding windows in time
                start_idx = j * self.N_step  # Start index of window
                end_idx = start_idx + self.N_win  # End index of window
                if end_idx > self.T:  # Avoid exceeding time dimension
                    break
                window_data = X[i, start_idx:end_idx]  # Extract window of size N_win

                # Step 4: Count peaks in the window
                F_peak[i, j] = self.count_peaks(window_data)

                # Step 5–6: Calculate energy feature
                energy_sum = 0
                for k in range(self.N_win - 2):  
                    if k + self.alpha < len(window_data):
                        energy_sum += (window_data[k + self.alpha] - window_data[k]) ** 2

                F_energy[i, j] = energy_sum / (self.T * self.X_base)

                # Step 8: Apply threshold to energy matrix
                if F_peak[i, j] > self.beta:
                    F_energy[i, j] = 1

        # Initialize aggregated matrices with slightly larger sizes
        num_windows_space = (self.L - self.N_d_point) // self.N_d_point + 3
        num_windows_combined = (self.num_windows - self.N_d_win) // self.N_d_win + 2

        M_peak = np.zeros((num_windows_space, num_windows_combined))
        M_energy = np.zeros((num_windows_space, num_windows_combined))

        # Step 11–15: Aggregating F_peak and F_energy in spatial and temporal windows
        for m in range(self.N_d_point // 2, self.L - self.N_d_point // 2):  # Spatial sliding window
            for n in range(0, self.num_windows - self.N_d_win + 1):  # Temporal sliding window
                spatial_idx = (m - self.N_d_point // 2) // self.N_d_point
                temporal_idx = n // self.N_d_win

                # Aggregating F_peak and F_energy
                M_peak[spatial_idx, temporal_idx] = np.mean(F_peak[m - self.N_d_point // 2 : m + self.N_d_point // 2 + 1, 
                                                                n : n + self.N_d_win])
                M_energy[spatial_idx, temporal_idx] = np.mean(F_energy[m - self.N_d_point // 2 : m + self.N_d_point // 2 + 1, 
                                                                        n : n + self.N_d_win])
        
        # Step to remove rows and columns where all data after them are zero
        M_peak = self.trim_zero_rows_and_cols(M_peak)
        M_energy = self.trim_zero_rows_and_cols(M_energy)

        return M_peak, M_energy

    def trim_zero_rows_and_cols(self, matrix):
        """
        Trim rows and columns where all subsequent data are zero.

        Parameters:
            matrix (ndarray): Input matrix to trim.
        
        Returns:
            trimmed_matrix (ndarray): Matrix after removing unnecessary rows and columns.
        """
        # Find the last non-zero row
        non_zero_rows = np.any(matrix != 0, axis=1)
        last_non_zero_row = np.where(non_zero_rows)[0].max() + 1 if non_zero_rows.any() else 0

        # Find the last non-zero column
        non_zero_cols = np.any(matrix != 0, axis=0)
        last_non_zero_col = np.where(non_zero_cols)[0].max() + 1 if non_zero_cols.any() else 0

        # Trim the matrix
        trimmed_matrix = matrix[:last_non_zero_row, :last_non_zero_col]

        return trimmed_matrix
